### Task1:Detecting Faces and Eyes
Use Haar Cascades to dectect your faces and eyes

Import necessary libraries

In [5]:
#Use Javascript code to call your local webcam
from IPython.display import display, Javascript
from google.colab.output import eval_js
#Convert the image type that captured by webcam
from base64 import b64decode, b64encode

import numpy as np
from PIL import Image

import io
import cv2 # OpenCV library 

JavaScript Code to call local WebCam and the image type is base64

In [6]:
from google.colab.output import eval_js

def VideoCapture():
  js = Javascript('''
    async function create(){
      div = document.createElement('div');
      document.body.appendChild(div);

      video = document.createElement('video');
      video.setAttribute('playsinline', '');

      div.appendChild(video);
      stream = await navigator.mediaDevices.getUserMedia({video: {facingMode: "environment"}});
      video.srcObject = stream;

      await video.play();

      canvas =  document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);

      div_out = document.createElement('div');
      document.body.appendChild(div_out);
      img = document.createElement('img');
      div_out.appendChild(img);
    }

    async function capture(){
        return await new Promise(function(resolve, reject){
            pendingResolve = resolve;
            canvas.getContext('2d').drawImage(video, 0, 0);
            result = canvas.toDataURL('image/jpeg', 0.30);

            pendingResolve(result);
        })
    }

    function showimg(imgb64){
        img.src = "data:image/jpg;base64," + imgb64;
    }

  ''')
  display(js)

Define the Image Conversion function to convert photo type 

For VideoCapture functions return images with base64 type

For cv2 need images with bytes type

In [7]:
def b64_to_bytes(byte):
  jpeg = b64decode(byte.split(',')[1])
  im = Image.open(io.BytesIO(jpeg))
  return np.array(im)

def bytes_to_b64(image):
  image = Image.fromarray(image)
  buffer = io.BytesIO()
  image.save(buffer, 'jpeg')
  buffer.seek(0)
  x = b64encode(buffer.read()).decode('utf-8')
  return x

The haar cascades files are in Google drive

In [8]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
# Load cascades using cv2.CascadeClassifier
faces_cascade = cv2.CascadeClassifier("/content/drive/MyDrive/Computer_Vision/haarcascade_frontalface_default.xml")
eyes_cascade = cv2.CascadeClassifier("/content/drive/MyDrive/Computer_Vision/haarcascade_eye.xml")

In [ ]:
VideoCapture()
# call the local camera to capture
eval_js('create()')

while True:
  b64_im = eval_js('capture()') # Take a capture 
  im = b64_to_bytes(b64_im) # Convert the capture brom base64 to bytes array
  
  # convert the image into gray scale
  gray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)

  # detect faces which is a array(bottom left point x , y, faces width, faces height). Here we may detect multiple faces, so we use for loop to draw all of them
  faces = faces_cascade.detectMultiScale(gray, 1.3, 4)
  # loop faces array to draw rectangle
  for (x, y, w, h) in faces: 
    # draw a rectangle around the face
    cv2.rectangle(im,(x,y),(x+w,y+h),(0,255,0),2)
    # retrive th face sub region (gray and colored)
    # here dont be conused with the order of x and y
    # think the array slice, first is rows, and second is columns
    face_roi_gray = gray[y:y+h, x:x+w]
    face_roi_im = im[y:y+h, x:x+w]

    # detect eyes
    eyes = eyes_cascade.detectMultiScale(face_roi_gray, 1.3, 4)
    for(e_x,e_y,e_w,e_h)in eyes:
      #draw a rectangle around each eye
      cv2.rectangle(face_roi_im,(e_x,e_y),(e_x+e_w,e_y+e_h),(255,0,0),2)

  eval_js('showimg("{}")'.format(bytes_to_b64(im)))